In [1]:
%pip install findspark pymongo

Note: you may need to restart the kernel to use updated packages.


# Crate Directory and Output File

In [2]:
#Use '/resource' to syncronize folder with host

!mkdir -p ~/output/spark-model/stream-model

In [3]:
input_path  = '/home/jovyan/output/renamed-data/multi/FullDataResampling.csv'
output_path = '/home/jovyan/output/spark-model/stream-model'

# Import Package

In [4]:
import os
import time
import json
import pyspark
import findspark
from gridfs import GridFS
from datetime import datetime
from pymongo import MongoClient

from pyspark.ml.classification import DecisionTreeClassifier
# from pyspark.ml.classification import RandomForestClassifier

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from sklearn.metrics import confusion_matrix
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

findspark.init()

# DB Connections

In [5]:
client = MongoClient("mongodb://mongodb:27017")
db = client['mataelanglab']

result_col = db['spark_result']
cv_col = db['spark_cv']
model_col = GridFS(db, 'spark_model')

# Spark session & context

In [6]:
spark = (SparkSession
         .builder
         .master('local[2]')
         .appName('DecissionTree-AllFeature')
         .config("spark.executor.memory", "4g") #optional
         .config("spark.executor.cores","1") #optional
         .getOrCreate())
sc = spark.sparkContext

# Read Data

In [7]:
df = spark.read.csv(input_path, header="true", inferSchema =True)

# Normalize

In [8]:
df = df.select([F.col(column).cast('double') for column in df.columns])

# Classification Using All Feature

In [9]:
feature = [
    'flow_duration',
    'total_fwd_packet',
    'total_bwd_packets',
    'total_length_of_fwd_packet',
    'total_length_of_bwd_packet',
    'fwd_packet_length_max',
    'fwd_packet_length_min',
    'fwd_packet_length_mean',
    'fwd_packet_length_std',
    'bwd_packet_length_max',
    'bwd_packet_length_min',
    'bwd_packet_length_mean',
    'bwd_packet_length_std',
    'flow_bytes_per_s',
    'flow_packets_per_s',
    'flow_iat_mean',
    'flow_iat_std',
    'flow_iat_max',
    'flow_iat_min',
    'fwd_iat_total',
    'fwd_iat_mean',
    'fwd_iat_std',
    'fwd_iat_max',
    'fwd_iat_min',
    'bwd_iat_total',
    'bwd_iat_mean',
    'bwd_iat_std',
    'bwd_iat_max',
    'bwd_iat_min',
    'fwd_psh_flags',
    'bwd_psh_flags',
    'fwd_urg_flags',
    'bwd_urg_flags',
    'fwd_header_length',
    'bwd_header_length',
    'fwd_packets_per_s',
    'bwd_packets_per_s',
    'packet_length_min',
    'packet_length_max',
    'packet_length_mean',
    'packet_length_std',
    'packet_length_variance',
    'fin_flag_count',
    'syn_flag_count',
    'rst_flag_count',
    'psh_flag_count',
    'ack_flag_count',
    'urg_flag_count',
    'cwr_flag_count',
    'ece_flag_count',
    'down_per_up_ratio',
    'average_packet_size',
    'fwd_segment_size_avg',
    'bwd_segment_size_avg',
    'fwd_bytes_per_bulk_avg',
    'fwd_packet_per_bulk_avg',
    'fwd_bulk_rate_avg',
    'bwd_bytes_per_bulk_avg',
    'bwd_packet_per_bulk_avg',
    'bwd_bulk_rate_avg',
    'subflow_fwd_packets',
    'subflow_fwd_bytes',
    'subflow_bwd_packets',
    'subflow_bwd_bytes',
    'fwd_init_win_bytes',
    'bwd_init_win_bytes',
    'fwd_act_data_pkts',
    'fwd_seg_size_min',
    'active_mean',
    'active_std',
    'active_max',
    'active_min',
    'idle_mean',
    'idle_std',
    'idle_max',
    'idle_min',
]

# Pre-Processing

In [10]:
#Vector Assembler
vector_assembler = VectorAssembler(inputCols=feature, outputCol="SS_features")
df = vector_assembler.transform(df)

#Standard Scaler
scaler = StandardScaler(inputCol="SS_features", outputCol="scaledFeatures", withStd=True, withMean=False).fit(df)
df = scaler.transform(df)

# Processing

In [11]:
# Split the data
(training_data, test_data) = df.randomSplit([0.7,0.3], 42)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 10335
Test Dataset Count: 4289


In [12]:
start_time = time.time()

# Create Decision Tree classifer object
dt = DecisionTreeClassifier(featuresCol = 'scaledFeatures', labelCol = 'label')
# dt = RandomForestClassifier(labelCol="label", featuresCol="scaledFeatures")

# Train Decision Tree Classifer
dtModel = dt.fit(df)

duration = (time.time() - start_time)
print("--- %s seconds ---" % duration)

--- 7.949734926223755 seconds ---


In [13]:
start_time = time.time()

#Predict the response for test dataset
dt_predictions = dtModel.transform(test_data)

duration = (time.time() - start_time)
print("--- %s seconds ---" % duration)

--- 0.27204394340515137 seconds ---


In [14]:
# Calculate Confusion matrix
ypred = dt_predictions.select("prediction").collect()
ytest = dt_predictions.select("label").collect()

acc_eval    = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="accuracy")
recall_eval = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="weightedRecall")
prec_eval   = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="weightedPrecision")
f1_eval     = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="f1")

accuracy    = acc_eval.evaluate(dt_predictions)
recall      = recall_eval.evaluate(dt_predictions)
precision   = prec_eval.evaluate(dt_predictions)
f1_score    = f1_eval.evaluate(dt_predictions)

print("confusion matrix :",confusion_matrix(ytest, ypred).ravel())
print(str('accuracy :  %0.4f' % accuracy) +"\n" + str('recall : %0.4f' % recall) +
      "\n" + str('precision :  %0.4f' %precision) + "\n" + str('f1-measure : %0.4f' %f1_score))

confusion matrix : [471   2   0  14   0   0   5   9   3 535   2   0   0   0   0   0   2  10
 501   0  14   0   0   0  22   0   0 491   0   0   0   0   0   0   0   0
 570   0   0   0   0   0   0   0  49 501   0   0   0   0   0   0   0   0
 547   3   0   0   0   0   0   7  15 516]
accuracy :  0.9634
recall : 0.9634
precision :  0.9647
f1-measure : 0.9634


In [15]:
# Save Model
ml_path = output_path+"/model"
ss_path = output_path+"/standard-scaler"

!rm -rf $ml_path $ss_path

In [16]:
dtModel.save(ml_path)
scaler.save(ss_path)